# 67818 Applied Competitive Lab in Data Science

## Final Project

### Participants:

- **Name:** Dan Badur
- **Student ID:** 209019256
- 
- **Name:** Tzur Breen
- **Student ID:** 209354919
- 
- **Name:** Shir Elbilia
- **Student ID:** 208621102
- 
- **Name:** Eliya Hasson
- **Student ID:** 208845032
- 
- **Name:** Tal Barda
- **Student ID:** 208729210

#### 1. Data Preparation and Feature Engineering:

We will start by defining a function for calculating the distance between each point in our database and a set of polygons in a wanted shapefile.
The function is set to run on multiple cors for better performance. 

For our original data containing 571000 rows for an 8 core CPU (the code utilizes 0.75 of the cores):
Calculating for the nature polygons takes about 3 minutes.
For the others it's about 30 seconds to 1 minute each.

In [3]:
from tqdm import tqdm
import geopandas as gpd
import pandas as pd
import numpy as np
import concurrent.futures
import math
from distance_calculations import calculate_distances_chunk

data_df = pd.read_csv("data.csv")

def calculate_distances_gdf_to_polygon_parallel(points_gdf, polygons_gdf, num_cores, target_name, attribute_name=None):
    num_chunks = num_cores * 4  
    points_chunks = np.array_split(points_gdf, num_chunks)  
    
    with tqdm(total=len(points_chunks), desc="Processing chunks for "+target_name) as pbar:
        with concurrent.futures.ProcessPoolExecutor(max_workers=num_cores) as executor:
            futures = [executor.submit(calculate_distances_chunk, chunk.geometry, polygons_gdf, return_attribute=attribute_name) for chunk in points_chunks]
            
            results = []
            for future in concurrent.futures.as_completed(futures):
                results.extend(future.result())
                pbar.update(1)
    
    return results

import os
total_cores = os.cpu_count()
cores_to_use = math.ceil(total_cores * 0.75)
num_cores = max(1, cores_to_use)  #

data_gdf = gpd.GeoDataFrame(data_df, geometry=gpd.points_from_xy(data_df.LONGITUDE, data_df.LATITUDE), crs="EPSG:4326")

nature_gdf = gpd.read_file('Nature/Nature.shp')
powerline_gdf = gpd.read_file('Powerline/Powerline.shp')
camp_gdf = gpd.read_file('Campgrounds/Campgrounds.shp')
population_gdf = gpd.read_file('Population/Population.shp')
#city_gdf =  gpd.read_file('City/City.shp')

nature_gdf = nature_gdf.to_crs("EPSG:4326")
powerline_gdf = powerline_gdf.to_crs("EPSG:4326")
camp_gdf = camp_gdf.to_crs("EPSG:4326")
population_gdf = population_gdf.to_crs("EPSG:4326")
#city_gdf = city_gdf.to_crs("EPSG:4326")

data_df['distance_to_nearest_nature_km'] = calculate_distances_gdf_to_polygon_parallel(data_gdf, nature_gdf, num_cores, "Nature")
data_df['distance_to_nearest_powerline_km'] = calculate_distances_gdf_to_polygon_parallel(data_gdf, powerline_gdf, num_cores, "Powerline")
data_df['distance_to_nearest_campground_km'] = calculate_distances_gdf_to_polygon_parallel(data_gdf, camp_gdf, num_cores, "Campground")

distance_from_population_and_class = calculate_distances_gdf_to_polygon_parallel(data_gdf, population_gdf, num_cores, "Population", "POP_CLASS")
data_df['distance_to_nearest_population'], data_df['nearest_population_class'] = zip(*distance_from_population_and_class)

# data_df['distance_to_nearest_city_km'] = calculate_distances_gdf_to_polygon_parallel(data_gdf, city_gdf, num_cores, "City")

data_df.to_csv('data_with_distances.csv', index=False)

/Users/talbarda/Desktop/FinalProject/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/talbarda/Desktop/FinalProject/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/talbarda/Desktop/FinalProject/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/talbarda/Desktop/FinalProject/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future vers

Now we will calculate the averages for each cause of fire for every feature we calculated above: 

In [6]:
import pandas as pd

# Load the dataset
df = pd.read_csv("data_with_distances.csv")

# Get unique values in the STAT_CAUSE_DESCR column
unique_causes = df['STAT_CAUSE_DESCR'].unique()

for cause in unique_causes:
    # Filter the dataframe for the current cause
    cause_df = df[df['STAT_CAUSE_DESCR'] == cause]

    # Calculate the average distances
    avg_nature_distance = cause_df['distance_to_nearest_nature_km'].mean()
    avg_powerline_distance = cause_df['distance_to_nearest_powerline_km'].mean()
    avg_camp_distance = cause_df['distance_to_nearest_campground_km'].mean()
    avg_population_distance = cause_df['distance_to_nearest_population'].mean()
    avg_city_population_class = cause_df['nearest_population_class'].mean()

    # Print the averages for the current cause
    print(f"Averages {cause}:")
    print(f"  Nature distance: {avg_nature_distance:.2f} km")
    print(f"  Powerline distance: {avg_powerline_distance:.2f} km")
    print(f"  Campground distance: {avg_camp_distance:.2f} km")
    print(f"  Nearest population distance: {avg_population_distance:.2f} km")
    print(f"  Nearest population class: {avg_city_population_class:.2f}")
    # print(f"  City: {avg_city_distance:.2f} km")
    print()

print()
df = pd.read_csv("data_with_distances.csv")
avg_city_distance_by_fire_size = df.groupby('FIRE_SIZE_CLASS')['distance_to_nearest_population'].mean()

print("Average distance from the nearest population for each FIRE_SIZE_CLASS:")
print(avg_city_distance_by_fire_size)

Averages Miscellaneous:
  Nature distance: 25.35 km
  Powerline distance: 4.54 km
  Campground distance: 101.88 km
  Nearest population distance: 5.04 km
  Nearest population class: 4.33

Averages Arson:
  Nature distance: 20.71 km
  Powerline distance: 4.77 km
  Campground distance: 89.57 km
  Nearest population distance: 6.05 km
  Nearest population class: 3.75

Averages Debris Burning:
  Nature distance: 25.00 km
  Powerline distance: 4.19 km
  Campground distance: 93.55 km
  Nearest population distance: 4.91 km
  Nearest population class: 3.80

Averages Smoking:
  Nature distance: 23.95 km
  Powerline distance: 5.12 km
  Campground distance: 97.11 km
  Nearest population distance: 4.85 km
  Nearest population class: 4.18

Averages Campfire:
  Nature distance: 12.29 km
  Powerline distance: 10.07 km
  Campground distance: 60.21 km
  Nearest population distance: 8.45 km
  Nearest population class: 3.60

Averages Equipment Use:
  Nature distance: 25.08 km
  Powerline distance: 4.52 km

Next we will explore time related features. 